In [ ]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [ ]:
import logging
import pandas as pd
from datetime import datetime
from general_functions.return_active_audiences import return_all_audiences
from general_functions.call_api_with_account_id import call_api_with_accountId, call_api_with_service_token
from general_functions.constants import return_api_url_prod, return_service_token

In [ ]:
def return_treatment_names(api_url, account_id, treatment_ids):
    if len(treatment_ids) == 0:
        return None
    treatment_ids = [str(i) for i in treatment_ids]
    treatments = call_api_with_accountId(
        f"{api_url}treatments/query",
        account_id,
        {"id": treatment_ids},
        logging,
    )
    if len(treatments) == 0:
        return None
    treatments = [entry["name"] for entry in treatments]
    return treatments

In [ ]:
api_url = "https://targeting.innkeepr.ai/api/"
path_to_save = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/DataChecks/controlGroupCoverage/{datetime.today().date()}-"
service_token = return_service_token()

In [ ]:
accounts = call_api_with_service_token(f"{api_url}core/accounts/query", service_token, {}, logging)

In [11]:
results = pd.DataFrame()
for acc_entry in accounts: 
    account_name = acc_entry["name"]
    account_id = acc_entry["id"]
    print(f"account_name = {account_name}")
    audiences = return_all_audiences(api_url, account_id, logging)
    if audiences.empty:
        print(f"No Audiences found for customer {account_name}")
        continue
    audiences = audiences[audiences["status"]=="active"]
    audiences = audiences[["id","name","config.controlGroupTreatments","source","type","config.model"]]
    audiences["account"] = account_name
    audiences = audiences[["account","id","name","config.controlGroupTreatments","source","type","config.model"]]
    audiences["cgTreatmentNames"] = audiences["config.controlGroupTreatments"].apply(lambda x: return_treatment_names(api_url, account_id, x))
    results = pd.concat([results, audiences], ignore_index=True)
results

In [12]:
results.to_csv(f"{path_to_save}-list-cg-treatments.csv", index=False)

In [13]:
nr_audiences = results["id"].nunique()
results["cg_treatment"] = results["config.controlGroupTreatments"].apply(lambda x: x[0] if x else None)
nr_audiences_with_cg = results[results["cg_treatment"].notnull()]["id"].nunique()
nr_audiences_without_cg = results[results["cg_treatment"].isnull()]["id"].nunique()
nr_audiences, nr_audiences_with_cg, nr_audiences_without_cg

In [14]:
seed_retargeting_audiences = results[results["type"].isin(["seed","retargeting"])]
seed_retargeting_audiences = seed_retargeting_audiences[seed_retargeting_audiences["account"].isin(["Innkeepr"])==False]
print(seed_retargeting_audiences["account"].unique())
nr_audiences_seeds = seed_retargeting_audiences["id"].nunique()
seed_retargeting_audiences["cg_treatment"] = seed_retargeting_audiences["config.controlGroupTreatments"].apply(lambda x: x[0] if x else None)
nr_audiences_with_cg_seeds = seed_retargeting_audiences[seed_retargeting_audiences["cg_treatment"].notnull()]["id"].nunique()
nr_audiences_without_cg_seeds = seed_retargeting_audiences[seed_retargeting_audiences["cg_treatment"].isnull()]["id"].nunique()
nr_audiences_seeds, nr_audiences_with_cg_seeds, nr_audiences_without_cg_seeds

In [15]:
nr_audiences_with_cg_seeds/nr_audiences_seeds

# Causal vs. Conversion Models

In [16]:
test = pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/innkeepr-checkup-functions/src/evaluations/get_modelproperites_per_audience/all-get-modelproperties-per-audience-2025-08-25 16:31:45.907309.csv")
test

In [17]:
import numpy as np
test["model_type"] = np.where(
    test["path"].str.contains("conversion"), "conversion", "causal"
)
test["model_type"].value_counts()

In [19]:
19/(99+19)